# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [13]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [14]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find the perfect venue in San Francisco for the Tech Innovation Conference that meets all the event criteria.

Action: Search the internet
Action Input: {"search_query": "venues for events in San Francisco"} 


Search results: Title: Party Venues in San Francisco, CA - Eventective
Link: https://www.eventective.com/san-francisco-ca/party-event-venues/
Snippet: There are 415 Party Venues in and around San Francisco at Eventective.com. Use the filter options to find your Perfect San Francisco Party Venue.
---
Title: Top 20 Venues in San Francisco, CA - PartySlate
Link: https://www.partyslate.com/find-venues/near/san-francisco-ca-usa
Snippet: Top Venues in San Francisco ; San Francisco Design Center Galleria. 101 Henry Adams St, San Francisco, CA 94103, USA. Spaces for 750 Guests ; Pi

This is the agent final answer: Venue: City View at METREON
Location: SoMa
Capacity: 2,000
Website: www.cityviewmetreon.com

City View at Metreon in San Francisco is a versatile event venue. Newly renovated to offer both flexibility and upscale ambiance, this San Francisco event space features more than 31,000 square feet of space that can be adjusted to accommodate various events, ranging from intimate gatherings of 200 to large-scale events for up to 2,000 people. Its key attractions include floor-to-ceiling windows providing an unobstructed view of San Francisco’s skyline and an expansive outdoor terrace, offering a unique open-air experience. This makes it an ideal venue for receptions, conferences, and other special occasions.
Please provide a feedback: This is good
Final Answer: 
Venue: City View at METREON
Location: SoMa
Capacity: 2,000
Website: www.cityviewmetreon.com

City View at Metreon in San Francisco is a versatile event venue. Newly renovated to offer both flexibility an

 


Search results: Title: Soar in 2024: 50 Genius Event Marketing Hacks for Tech Companies
Link: https://www.linkedin.com/pulse/soar-2024-50-genius-event-marketing-hacks-tech-companies-mxqkc
Snippet: 50 Genius Event Marketing Hacks for Tech Companies to make events SOAR in 2024!
---
Title: Digital Event Marketing Strategies: Start Strong; Finish First
Link: https://www.smartmeetings.com/magazine_article/start-strong-finish-first-event-marketing
Snippet: Event marketing strategies and post-event actions are critical for increasing the success of digital events. Here are some best practices.
---
Title: The 2024 Guide to Your Event Marketing Strategy - Contrast
Link: https://www.getcontrast.io/learn/event-marketing-strategies
Snippet: Build the ultimate event marketing strategy with this guide. This article will cover all the key strategies for you to have a successful ...
---
Title: The #1 Event Marketing Strategy Guide For 2024 - Kaltura
Link: https://corp.kaltura.com/blog/event-market

Exception in thread Thread-5 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/us

 


Search results: Title: Soar in 2024: 50 Genius Event Marketing Hacks for Tech Companies
Link: https://www.linkedin.com/pulse/soar-2024-50-genius-event-marketing-hacks-tech-companies-mxqkc
Snippet: 50 Genius Event Marketing Hacks for Tech Companies to make events SOAR in 2024!
---
Title: 10 Effective Conference Marketing Ideas for 2024 | InEvent
Link: https://inevent.com/blog/others/conference-marketing-ideas.html
Snippet: Discover 10 innovative conference marketing ideas for 2024 to boost attendance and engagement, featuring digital strategies, personalization ...
---
Title: Innovative Event Marketing Strategies to Boost Corporate and Social ...
Link: https://www.linkedin.com/pulse/innovative-event-marketing-strategies-boost-corporate-b70rc
Snippet: The advancement of technology has opened new avenues for creativity and innovation in event marketing. From social media platforms to ...
---
Title: 11 Online Event Marketing Strategies for 2024 | Mighty Networks
Link: https://www.might

Thought: I now can give a great answer
Final Answer: Based on the search results, the City View at Metreon venue does not have valet parking, but it is centrally located in the heart of SoMa with close proximity to public parking and transit options. There are several parking options near the venue that can be reserved for guaranteed parking. Additionally, there are various transportation options available from San Francisco to Metreon, including bus, tram, taxi, and foot. The Tech Innovation Conference logistics can include providing this information to attendees to ensure smooth transportation and parking arrangements.

> Finished chain.
 

Based on the search results, the City View at Metreon venue does not have valet parking, but it is centrally located in the heart of SoMa with close proximity to public parking and transit options. There are several parking options near the venue that can be reserved for guaranteed parking. Additionally, there are various transportation options av

- Display the generated `venue_details.json` file.

In [15]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'SoMa',
 'booking_status': 'Available',
 'capacity': 2000,
 'name': 'City View at METREON'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [16]:
from IPython.display import Markdown
Markdown("marketing_report.md")

marketing_report.md